In [12]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
from skimage import color
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA as RandomizedPCA
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras import optimizers


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


###############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=10, resize=1)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

print(lfw_people.images.shape)
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.images
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target

target_names = lfw_people.target_names
n_classes = target_names.shape[0]
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Automatically created module for IPython interactive environment
(4324, 125, 94)
Total dataset size:
n_samples: 4324
n_features: 125
n_classes: 158


some preprocessing

In [13]:
X_3chan = color.grey2rgb(X)
y = to_categorical(y, num_classes=n_classes)

Split into test and training sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X_3chan, y, test_size=0.25)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3243, 125, 94, 3)
(3243, 158)
(1081, 125, 94, 3)
(1081, 158)


Initialize base model

In [15]:
base_model = InceptionV3(weights=None, include_top=False, input_shape=(h,w,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

BS = 32
EPOCHS = 100

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=BS),
                     steps_per_epoch=len(X_train) / BS, epochs=EPOCHS)

# train the model on the new data for a few epochs
#model.fit(X_train, y_train, batch_size=BS, epochs=EPOCHS)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adagrad(lr=0.01, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=BS, epochs=EPOCHS // 10)

model.evaluate(X_test, y_test)


Epoch 1/100
102/101 [==============================] - 67s 657ms/step - loss: 5.0432 - acc: 0.1005
Epoch 2/100
102/101 [==============================] - 48s 472ms/step - loss: 4.6183 - acc: 0.1131
Epoch 3/100
102/101 [==============================] - 56s 552ms/step - loss: 4.5803 - acc: 0.1155
Epoch 4/100
102/101 [==============================] - 61s 596ms/step - loss: 4.5436 - acc: 0.1191
Epoch 5/100
102/101 [==============================] - 49s 480ms/step - loss: 4.5224 - acc: 0.1185
Epoch 6/100
102/101 [==============================] - 52s 514ms/step - loss: 4.5031 - acc: 0.1194
Epoch 7/100
102/101 [==============================] - 50s 487ms/step - loss: 4.4708 - acc: 0.1225
Epoch 8/100
102/101 [==============================] - 46s 453ms/step - loss: 4.4610 - acc: 0.1201
Epoch 9/100
102/101 [==============================] - 47s 458ms/step - loss: 4.4251 - acc: 0.1167
Epoch 10/100
102/101 [==============================] - 49s 477ms/step - loss: 4.4384 - acc: 0.1195
Epoch 11/

[4.770985063416996, 0.1295097131457845]